In [1]:
import json
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import nltk

In [2]:
with open('data/corpus.json') as f:
    corpus = json.load(f)

In [3]:
'Текстов - {}, слов - {}'.format(len(corpus), sum([len(sample) for sample in corpus]))

'Текстов - 1157366, слов - 16028874'

In [4]:
morph = MorphAnalyzer()

In [5]:
def get_lemma(word):
    
    word_data = morph.parse(word)[0]
    
    return word_data.normal_form

## Соберем словарь лем
В нашем корпусе 16028874 слов. Лемматизировать весь корпус будет очень долго. Давайте лучше соберем словарь уникальных слов и будет лемматизировать только уникальные слова.

In [6]:
tok2lemma = {}

for text in tqdm(corpus):
    for tok in text:
        if tok not in tok2lemma:
            tok2lemma[tok] = get_lemma(tok)

100%|██████████| 1157366/1157366 [00:38<00:00, 30225.70it/s]


In [7]:
# ключ - уникальное слово в нашем корпусе, значение - его лемма
tok2lemma['уехали']

'уехать'

In [8]:
len(tok2lemma)

193435

In [9]:
stopwords = nltk.corpus.stopwords.words('russian')

# Замена слов леммами
Так как теперь к каждому слову из нашего корпуса мы знаем лемму, то давайте каждое слово заменим на его лемму и уберем стоп слова.
Это работает гораздо(!) быстрее, чем если бы мы в корпусе для каждого слова каждый раз рассчитывали лемму (с помощью пайморфи), 
потому что теперь нам надо вызвать пайморфи 193435 раз вместо 16028874.

In [10]:
lemmas_corpus = [[tok2lemma[tok] for tok in text if tok not in stopwords and tok]
                 for text in tqdm(corpus)]

100%|██████████| 1157366/1157366 [00:24<00:00, 46555.20it/s]


# Соберем частотный словарь

In [11]:
freq = {}

for text in tqdm(lemmas_corpus):
    for tok in text:
        if tok in freq:
            freq[tok] += 1
        else:
            freq[tok] = 1

100%|██████████| 1157366/1157366 [00:02<00:00, 395416.91it/s]


In [12]:
freq_df = pd.DataFrame(data={'word': list(freq.keys()), 'n_entries': list(freq.values())})

In [13]:
freq_df.sort_values(by=['n_entries'], ascending=False, inplace=True)

In [14]:
freq_df.head()

,word,n_entries
21,##число,413016
3,банка,197884
47,карта,156216
25,банк,135943
134,кредит,86865


In [15]:
# уникальных слов в словаре
freq_df.shape

(77002, 2)

In [16]:
freq_df.tail()

,word,n_entries
45645,пролом,1
45647,запачкать,1
45648,гих,1
45650,поcтоянно,1
77001,ситуевина,1


In [17]:
n_words = freq_df.n_entries.sum()

# Замена редких слов
В нашем корпусе осталось много слов, которые встречаются очень редко. Давайте мы редкие слова заменим на специальный токе UNK - unknown. Так мы разительно сократим размер нашего словаря слов с незначительной потерей информации.

In [18]:
print('Доля слов, которые мы заменим на UNK:')

for threshold in np.arange(5, 36, 5):
    
    sub_df = freq_df[freq_df.n_entries < threshold]
    
    unk_freq = sub_df['n_entries'].sum() * 100 / n_words
    
    print('Порог отсечения - {}, доля UNK - {:.2f} %, слов в слове - {}, удалили - {} слов'.format(
        threshold, unk_freq, freq_df.shape[0] - sub_df.shape[0], sub_df.shape[0]))

Доля слов, которые мы заменим на UNK:
Порог отсечения - 5, доля UNK - 0.76 %, слов в слове - 23389, удалили - 53613 слов
Порог отсечения - 10, доля UNK - 1.18 %, слов в слове - 16851, удалили - 60151 слов
Порог отсечения - 15, доля UNK - 1.51 %, слов в слове - 13861, удалили - 63141 слов
Порог отсечения - 20, доля UNK - 1.82 %, слов в слове - 12010, удалили - 64992 слов
Порог отсечения - 25, доля UNK - 2.07 %, слов в слове - 10798, удалили - 66204 слов
Порог отсечения - 30, доля UNK - 2.31 %, слов в слове - 9882, удалили - 67120 слов
Порог отсечения - 35, доля UNK - 2.53 %, слов в слове - 9170, удалили - 67832 слов


In [19]:
# кажется, что оптимально, но обычно берут меньше
threshold = 15

In [20]:
vocab = freq_df[freq_df.n_entries >= threshold]

In [21]:
words = set(vocab.word)

In [22]:
len(words)

13861

In [23]:
'Мы сократили наш словарь в {:.2f} раз с потерей 1.51 % всех слов'.format(freq_df.shape[0] / len(words))

'Мы сократили наш словарь в 5.56 раз с потерей 1.51 % всех слов'

In [24]:
def get_correct_words(word):
    
    if word in words:
        return word
    else:
        return 'UNK'

In [25]:
# заменим слово токеном UNK, если его нет в нашем новом словаре
processed_corpus = [[get_correct_words(tok) for tok in text] for text in tqdm(lemmas_corpus)]

100%|██████████| 1157366/1157366 [00:04<00:00, 265597.79it/s]


In [26]:
def drop_duplicate_unks(tokens):
    
    output_tokens = []
    
    for tok in tokens:
        
        if tok == 'UNK' and output_tokens and output_tokens[-1] == 'UNK':
            continue
            
        output_tokens.append(tok)
            
    return output_tokens

In [27]:
sample_text = 'думать далее милый барышня UNK UNK тинькоф звонить неделя'.split()

In [28]:
sample_text

['думать',
 'далее',
 'милый',
 'барышня',
 'UNK',
 'UNK',
 'тинькоф',
 'звонить',
 'неделя']

In [29]:
drop_duplicate_unks(sample_text)

['думать', 'далее', 'милый', 'барышня', 'UNK', 'тинькоф', 'звонить', 'неделя']

In [30]:
# дедублируем подряд идущие унки (оставим только один)
processed_corpus = [drop_duplicate_unks(sample) for sample in tqdm(processed_corpus)]

100%|██████████| 1157366/1157366 [00:03<00:00, 348464.74it/s]


In [31]:
texts_with_unk = [text for text in processed_corpus if 'UNK' in text]
'Текстов с унками - {:.2f} %'.format(len(texts_with_unk) * 100 / len(processed_corpus))

'Текстов с унками - 11.28 %'

In [32]:
# посмотрим на тексты с унками
for text in random.sample(texts_with_unk, k=5):
    print(' '.join(text))

нужный международный стандарт UNK знать такой бред этот momentum
сегодня посетить филиал банка UNK хотеть узнать кредитный карта условие
стать хороший UNK ваш софт UNK рынок весь счастие
норматив рассмотрение заявка ##число день комиссия вообще UNK четверг
обслуживать шаг ва UNK приятно удивить отношение сотрудник банка клиент


In [33]:
# выглядит не так плохо

In [34]:
random.shuffle(processed_corpus)

# Выберем подвыборку данных
Чтобы быстрее выучить word2vec

In [35]:
sub_data = processed_corpus[:100000]

In [36]:
with open('data/processed_corpus.json', 'w') as f:
    json.dump(sub_data, f, ensure_ascii=False)